## Week 2 - Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```

In [ ]:
# Install or upgrade necessary libraries:
# - fastembed: library for generating text embeddings
# - qdrant-client with fastembed extension: client for Qdrant vector database 
# - sentence-transformers: library for computing dense vector representations
# The -q flag makes the installation output less verbose
!pip install --upgrade fastembed "qdrant-client[fastembed]>=1.14.2" sentence-transformers -q

In [2]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np
import requests

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51


In [46]:
#docker pull qdrant/qdrant
#docker run -p 6333:6333 -p 6334:6334 -v "$(pwd)/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

In [3]:
query = 'I just discovered the course. Can I join now?'
model_name = 'jinaai/jina-embeddings-v2-small-en'

model = TextEmbedding(model_name=model_name)
embeddings_query = list(model.embed([query]))
len(embeddings_query[0])

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

512

In [ ]:
# Find the minimum value in the embedding vector and round it to 3 decimal places
# This answers Q1 which asks for the minimal value in the embedding array
round(np.min(embeddings_query[0]), 3)

np.float64(-0.117)

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

In [5]:
# Calculate the norm (length) of the embedding vector
norm = np.linalg.norm(embeddings_query[0])

# Calculate dot product of the vector with itself
dot_product = embeddings_query[0].dot(embeddings_query[0])

# Display both values to verify that the vector is normalized
# If normalized: norm ≈ 1.0 and dot_product ≈ 1.0 
norm, dot_product

(np.float64(1.0), np.float64(1.0000000000000002))

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9

In [6]:
# Define a document text that we want to compare with our query
doc = 'Can I still join the course after the start date?'

# Convert the document text into an embedding vector using the same model
embeddings_doc = list(model.embed([doc]))

# Calculate cosine similarity between query and document vectors
# Since both vectors are normalized, dot product equals cosine similarity
embeddings_doc[0].dot(embeddings_query[0])

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [7]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
              'section': 'General course-related questions',
              'question': 'Course - Can I still join the course after the start date?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
              'section': 'General course-related questions',
              'question': 'Course - Can I follow the course after it finishes?',
              'course': 'data-engineering-zoomcamp'},
             {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
              'section': 'General course-related questions',
              'question': 'Course - When will the course start?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
              'section': 'General course-related questions',
              'question': 'Course - What can I do before the course starts?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
              'section': 'General course-related questions',
              'question': 'How can we contribute to the course?',
              'course': 'data-engineering-zoomcamp'}
            ]

embeddings_vector = []

for document in documents:
    embeddings_doc = list(model.embed([document['text']]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmin(cos_sim)

np.int64(3)

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [8]:
embeddings_vector = []

for document in documents:
    full_text = document['question'] + ' ' + document['text']
    embeddings_doc = list(model.embed([full_text]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmax(cos_sim)

np.int64(0)

Yes, in general it can be different because:
+ In Q3 you are comparing only the body of the text.
+ In Q4, the `question` provides key information that improves the semantic similarity if the query is similar to the original question.

## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in `fastembed`?

- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [9]:
models_list = TextEmbedding.list_supported_models()

dimensions = []
for model in models_list:
    dimensions.append(model['dim'])
    
models_list[np.array(dimensions).argmin()]

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67

In [10]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
documents[:5]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [13]:
client = QdrantClient(":memory:")  # Use in-memory storage instead of connecting to a server

EMBEDDING_DIMENSIONALITY = 384
collection_name = "ml-zoomcamp-rag"
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

model_name = 'BAAI/bge-small-en'
points = []
id_ = 0
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id_,
        vector=models.Document(text=text, model=model_name),
        payload={
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        }
    )
    points.append(point)

    id_ += 1

client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

Bad pipe message: %s [b'"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand']
Bad pipe message: %s [b'v="24"\r\nsec-ch-ua-mobile: ?0\r\nsec', b'h-ua-platform: "Windows"\r\nUpgrade-Insecure-R', b'uests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Geck', b' Chrome/137.0.0.0 Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,imag', b'webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mo', b': navigate\r\nSec-Fetch-User: ?1\r\nSec-Fetch-Dest: document\r\nAccept-Encoding: gzip, deflate, br, zstd\r', b'ccept-Lan', b'age: ar-EG,ar;q=0.9,en-US;q=0.8,en;q=0.7\r\nCookie: adminer_permanent=; grafana_session=3540cfce971f6d61', b'736b2735647ccd; grafana_session_expiry=1750999677\r\n\r']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"\r\nsec-ch-ua-mobile: ?0\r\n']
Bad pipe message: %s [b'c-

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
def search(query, limit=5, score_threshold=0.0):
    """
    Search for documents in Qdrant vector database using semantic similarity.
    
    Args:
        query (str): The search query text
        limit (int): Maximum number of results to return
        score_threshold (float): Minimum relevance score (0.0-1.0) for results
        
    Returns:
        SearchResult object containing matching points
    """
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name  # Using BAAI/bge-small-en model
        ),
        limit=limit,
        score_threshold=score_threshold,
        with_payload=True
    )
    
    return results

# Get the search results and extract the top match
result = search(query, limit=1).points[0]

In [19]:
query

'I just discovered the course. Can I join now?'

In [20]:
result.payload["text"]

'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [21]:
result.score

0.8703173141122041